<a href="https://colab.research.google.com/github/grosetta/rdkit_study/blob/main/mcts_ligand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#文件头代码
import sys
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0,"/content/gdrive/My Drive/Colab Notebooks/python_package/")

Mounted at /content/gdrive


# 模块导入

In [8]:
from subprocess import Popen, PIPE
from math import *
import random
import random as pr
import numpy as np
from copy import deepcopy
#from types import IntType, ListType, TupleType, StringTypes
import itertools
import time
import math
import argparse
import subprocess
from keras.preprocessing import sequence
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
import sys
import csv
import itertools
import operator
import nltk
import os
from IPython import display
import matplotlib.pyplot as plt
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles, MolToSmiles
import pickle
import gzip
import networkx as nx
from rdkit.Chem import rdmolops
import h5py
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Activation,TimeDistributed
from keras.layers import LSTM,GRU
from tensorflow.python.keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.models import model_from_json
from __future__ import print_function
from rdkit.Chem import rdMolDescriptors
from rdkit.six.moves import cPickle
from rdkit.six import iteritems
from collections import defaultdict
import os.path as op

# 定义函数

In [23]:
def zinc_data_with_bracket_original():
#将文件的所有smile转存为一个列表，列表中每一个元素是一个smile
    sen_space=[]
    f = open('/content/gdrive/My Drive/data/250k_rndm_zinc_drugs_clean.smi', 'r')

    reader = csv.reader(f)
    for row in reader:
        sen_space.append(row) #此时的sen_space是列表里套列表的形式
    f.close()

    zinc_processed=[]
    for i in range(len(sen_space)):
        word1=sen_space[i]
        zinc_processed.append(word1[0])
    return zinc_processed

In [10]:
def zinc_processed_with_bracket(sen_space):
    #print sen_space
    all_smile=[]
    length=[]
    end="\n"
    element_table=["C","N","B","O","P","S","F","Cl","Br","I","(",")","=","#"]
    ring=["1","2","3","4","5","6","7","8","9","10"]

    for i in range(len(sen_space)):
        #word1=sen_space[i]
        word_space=sen_space[i]
        word=[]
        j=0
        while j<len(word_space):
            word_space1=[]
            if word_space[j]=="[":
                word_space1.append(word_space[j])
                j=j+1
                while word_space[j]!="]":
                    word_space1.append(word_space[j])
                    j=j+1
                word_space1.append(word_space[j])
                word_space2=''.join(word_space1)
                word.append(word_space2)
                j=j+1
            else:
                word_space1.append(word_space[j])

                if j+1<len(word_space):
                    word_space1.append(word_space[j+1])
                    word_space2=''.join(word_space1)
                else:
                    word_space1.insert(0,word_space[j-1])
                    word_space2=''.join(word_space1)

                if word_space2 not in element_table:
                    word.append(word_space[j])
                    j=j+1
                else:
                    word.append(word_space2)
                    j=j+2


        word.append(end)
        word.insert(0,"&")
        len1=len(word)
        length.append(len1)
        all_smile.append(list(word))
    val=["\n"]
    for i in range(len(all_smile)):
        for j in range(len(all_smile[i])):
            if all_smile[i][j] not in val:
                val.append(all_smile[i][j])

    return val, all_smile

In [11]:
def loaded_model():

    json_file = open('/content/gdrive/My Drive/data/RNN_model/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights('/content/gdrive/My Drive/data/RNN_model/model.h5')
    print("Loaded model from disk")

In [12]:
class chemical:

    def __init__(self):

        self.position=['&']
        self.num_atom=8
        self.vl=['\n', '&', 'C', '(', 'c', '1', 'o', '=', 'O', 'N', 'F', '[C@@H]',
        'n', '-', '#', 'S', 'Cl', '[O-]', '[C@H]', '[NH+]', '[C@]', 's', 'Br', '/', '[nH]', '[NH3+]',
     '[NH2+]', '[C@@]', '[N+]', '[nH+]', '\\', '[S@]', '[N-]', '[n+]', '[S@@]', '[S-]',
         'I', '[n-]', 'P', '[OH+]', '[NH-]', '[P@@H]', '[P@@]', '[PH2]', '[P@]', '[P+]', '[S+]',
        '[o+]', '[CH2-]', '[CH-]', '[SH+]', '[O+]', '[s+]', '[PH+]', '[PH]', '[S@@+]']

    def Clone(self):

        st = chemical()
        st.position= self.position[:]
        return st

    def SelectPosition(self,m):
        self.position.append(m)

    def Getatom(self):
        return [i for i in range(self.num_atom)]

In [30]:
def rdock_score(compound):

    input_smiles = str(compound)
    num_docking = 3 # number of docking trials
    score_name = '<SCORE>' # <SCORE> or <SCORE.INTER>

    sdf_file = 'tmp_comp.sdf'
    docking_result_file = 'rdock_out'
    min_score = 10**10

    #----Translation from SMILES to sdf
    fw = Chem.SDWriter(sdf_file)
    m1 = Chem.MolFromSmiles(input_smiles)
    try:
        if m1!= None:
            m = Chem.AddHs(m1)

            cid = AllChem.EmbedMolecule(m)

            opt = AllChem.UFFOptimizeMolecule(m,maxIters=200)
            print(opt)

            fw.write(m)
            fw.close()

            #----rdock calculation
            cmd = '$RBT_ROOT/build/exe/rbdock -r cavity.prm -p $RBT_ROOT/data/scripts/dock.prm -i ' + sdf_file + ' -o ' + docking_result_file + ' -T 1 -n' + str(num_docking)
            proc = subprocess.call( cmd , shell=True)

            #----find the minimum score of rdock from multiple docking results
            f = open(docking_result_file+'.sd')
            lines = f.readlines()
            f.close()

            line_count = 0
            score_line = -1
            for line in lines:
                v_list = line.split()
                if line_count == score_line:
                    if float(v_list[0]) < min_score:
                        min_score = float(v_list[0])

                if len(v_list) <= 1:
                    line_count += 1
                    continue

                if v_list[1] == score_name:
                    score_line = line_count + 1

                line_count += 1
            print('minimum rdock score', min_score)
    except:
        min_score=10**10

    return min_score

In [29]:
def expanded_node(model,state,val):

    all_nodes=[]

    end="\n"
    position=[]
    position.extend(state)
    total_generated=[]
    new_compound=[]
    get_int_old=[]
    for j in range(len(position)):
        get_int_old.append(val.index(position[j]))

    get_int=get_int_old

    x=np.reshape(get_int,(1,len(get_int)))
    x_pad= sequence.pad_sequences(x, maxlen=82, dtype='int32',
        padding='post', truncating='pre', value=0.)

    for i in range(30):
        predictions=model.predict(x_pad)
        #print "shape of RNN",predictions.shape
        preds=np.asarray(predictions[0][len(get_int)-1]).astype('float64')
        preds = np.log(preds) / 1.0
        preds = np.exp(preds) / np.sum(np.exp(preds))
        next_probas = np.random.multinomial(1, preds, 1)
        next_int=np.argmax(next_probas)
        all_nodes.append(next_int)

    all_nodes=list(set(all_nodes))
    print(all_nodes)
    return all_nodes
##########################################################################################
def node_to_add(all_nodes,val):
    added_nodes=[]
    for i in range(len(all_nodes)):
        added_nodes.append(val[all_nodes[i]])

    print(added_nodes)

    return added_nodes
#######################################################################################
def chem_kn_simulation(model,state,val,added_nodes):
    all_posible=[]

    end="\n"
    for i in range(len(added_nodes)):
        position=[]
        position.extend(state)
        position.append(added_nodes[i])
        total_generated=[]
        new_compound=[]
        get_int_old=[]
        for j in range(len(position)):
            get_int_old.append(val.index(position[j]))

        get_int=get_int_old

        x=np.reshape(get_int,(1,len(get_int)))
        x_pad= sequence.pad_sequences(x, maxlen=82, dtype='int32',
            padding='post', truncating='pre', value=0.)
        while not get_int[-1] == val.index(end):
            predictions=model.predict(x_pad)
            #print "shape of RNN",predictions.shape
            preds=np.asarray(predictions[0][len(get_int)-1]).astype('float64')
            preds = np.log(preds) / 1.0
            preds = np.exp(preds) / np.sum(np.exp(preds))
            next_probas = np.random.multinomial(1, preds, 1)
            #print predictions[0][len(get_int)-1]
            #print "next probas",next_probas
            next_int=np.argmax(next_probas)
            a=predictions[0][len(get_int)-1]
            next_int_test=sorted(range(len(a)), key=lambda i: a[i])[-10:]
            get_int.append(next_int)
            x=np.reshape(get_int,(1,len(get_int)))
            x_pad = sequence.pad_sequences(x, maxlen=82, dtype='int32',
                padding='post', truncating='pre', value=0.)
            if len(get_int)>82:
                break
        total_generated.append(get_int)
        all_posible.extend(total_generated)

    return all_posible
#################################################################################################
def predict_smile(all_posible,val):


    new_compound=[]
    for i in range(len(all_posible)):
        total_generated=all_posible[i]

        generate_smile=[]

        for j in range(len(total_generated)-1):
            generate_smile.append(val[total_generated[j]])
        generate_smile.remove("&")
        new_compound.append(generate_smile)

    return new_compound
###############################################################################################
def make_input_smile(generate_smile):
    new_compound=[]
    for i in range(len(generate_smile)):
        middle=[]
        for j in range(len(generate_smile[i])):
            middle.append(generate_smile[i][j])
        com=''.join(middle)
        new_compound.append(com)

    return new_compound
###############################################################################################
def check_node_type(new_compound):
    node_index=[]
    valid_compound=[]
    all_smile=[]
    distance=[]

    score=[]
    for i in range(len(new_compound)):
        try:
            ko = Chem.MolFromSmiles(new_compound[i])
            
        except:
            ko=None
        
        if ko!=None:
            try:
                molscore=MolFromSmiles(new_compound[i])
            except:
                molscore=None
            if molscore!=None:
                SA_score = -sascorer.calculateScore(molscore)
            else:
                SA_score=1000
            cycle_list = nx.cycle_basis(nx.Graph(rdmolops.GetAdjacencyMatrix(MolFromSmiles(new_compound[i]))))
            if len(cycle_list) == 0:
                 cycle_length =0
            else:
                cycle_length = max([ len(j) for j in cycle_list ])
            if cycle_length <= 6:
                cycle_length = 0
            if cycle_length==0:
                m=rdock_score(new_compound[i])
                if m<10**10:
                    node_index.append(i)
                    valid_compound.append(new_compound[i])
                    score.append(m)
    return node_index,score,valid_compound

In [ ]:
class Node:

    def __init__(self, position = None,  parent = None, state = None):
        self.position = position
        self.parentNode = parent
        self.childNodes = []
        self.child=None
        self.wins = 0
        self.visits = 0
        self.nonvisited_atom=state.Getatom()
        self.type_node=[]
        self.depth=0

    def Selectnode(self):

        ucb=[]
        for i in range(len(self.childNodes)):
            ucb.append(self.childNodes[i].wins/self.childNodes[i].visits+sqrt(2)*sqrt(2*log(self.visits)/self.childNodes[i].visits))
        m = np.amax(ucb)
        indices = np.nonzero(ucb == m)[0]
        ind=pr.choice(indices)
        s=self.childNodes[ind]

        return s

    def Addnode(self, m, s):

        n = Node(position = m, parent = self, state = s)
        self.childNodes.append(n)

    def simulation(self,state):
        predicted_smile=predict_smile(model,state)
        input_smile=make_input_smile(predicted_smile)
        logp,valid_smile,all_smile=logp_calculation(input_smile)

        return logp,valid_smile,all_smile

    def Update(self, result):

        self.visits += 1
        self.wins += result

In [16]:
def MCTS(root, verbose = False):

    """initialization of the chemical trees and grammar trees"""
    run_time=time.time()+3600*48
    rootnode = Node(state = root)
    state = root.Clone()
    """----------------------------------------------------------------------"""


    """global variables used for save valid compounds and simulated compounds"""
    valid_compound=[]
    all_simulated_compound=[]
    desired_compound=[]
    max_logp=[]
    desired_activity=[]
    depth=[]
    min_score=1000
    score_distribution=[]
    min_score_distribution=[]

    """----------------------------------------------------------------------"""

    while time.time()<=run_time:

        node = rootnode # important !    this node is different with state / node is the tree node
        state = root.Clone() # but this state is the state of the initialization .  too important !!!
        """selection step"""
        node_pool=[]

        while node.childNodes!=[]:
            node = node.Selectnode()
            state.SelectPosition(node.position)
        print("state position:,",state.position)


        """------------------------------------------------------------------"""

        """expansion step"""
        expanded=expanded_node(model,state.position,val)
        nodeadded=node_to_add(expanded,val)
        all_posible=chem_kn_simulation(model,state.position,val,nodeadded)
        generate_smile=predict_smile(all_posible,val)
        new_compound=make_input_smile(generate_smile)



        node_index,rdock_score,valid_smile=check_node_type(new_compound)
        valid_compound.extend(valid_smile)
        score_distribution.extend(rdock_score)

        if len(node_index)==0:
            re=-1.0
            while node != None:
                node.Update(re)
                node = node.parentNode
        else:
            re=[]
            for i in range(len(node_index)):
                m=node_index[i]
                node.Addnode(nodeadded[m],state)
                node_pool.append(node.childNodes[i])
                depth.append(len(state.position))
                print("current minmum score",min_score)
                if rdock_score[i]<=min_score:
                    min_score_distribution.append(rdock_score[i])
                    min_score=rdock_score[i]
                else:
                    min_score_distribution.append(min_score)
                """simulation"""
                re.append((-0.8*rdock_score[i])/(1+0.8*abs(rdock_score[i])))
                """backpropation step"""

            for i in range(len(node_pool)):

                node=node_pool[i]
                while node != None:
                    node.Update(re[i])
                    node = node.parentNode




        """check if found the desired compound"""

    print("rdock_score",score_distribution)
    print("num valid_compound:",len(valid_compound))
    print("valid compounds",valid_compound)
    print("depth",depth)
    print("min_score",min_score_distribution)


    return valid_compound

In [17]:
def UCTchemical():
    one_search_start_time=time.time()
    time_out=one_search_start_time+60*10
    state = chemical()
    best = MCTS(root = state,verbose = False)


    return best

# 代码主体

In [ ]:
smile_old=zinc_data_with_bracket_original()
val,smile=zinc_processed_with_bracket(smile_old)
print(val)


model=loaded_model()
valid_compound=UCTchemical()